In [27]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import json
import pickle
import pandas as pd

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

In [28]:
with open("../data/models/season_models.pkl", "rb") as f:
    models = pickle.load(f)

In [37]:
#Load RF model  pickle Load sarima pickle then GPT decides which model to call

In [ ]:
def predict_response_time(season, borough, dispatch_area, initial_type,
                           hour, day, month, is_weekend, is_holiday,
                           is_rush_hour, initial_severity, zipcode,
                           temperature, precipitation, windspeed, weathercode,
                           closest_station_manhattan_miles,
                           special_events=0, standby=0, held=0):  # defaults to 0
    season = season.lower()
    model = models[season]
    features = model.feature_names_in_
    row = {f: 0 for f in features}
    row["closest_station_manhattan_miles"] = closest_station_manhattan_miles
    row["is_weekend"] = is_weekend
    row["hour"] = hour
    row["is_holiday"] = is_holiday
    row["is_rush_hour_1"] = is_rush_hour
    row["initial_severity"] = initial_severity
    row["zipcode"] = zipcode
    row["precipitation"] = precipitation
    row["windspeed"] = windspeed
    row["weathercode"] = weathercode
    row["day"] = day
    row["month"] = month
    row["temperture"]= temperature
    row["held_Y"] = held
    row["standby_Y"] = standby
    row["special_events_Y"] = special_events
    
    
    #one hot encode the categorical variables
    

    borough_key = f"borough_{borough.upper()}"
    if borough_key in row:
        row[borough_key] = 1
    
    dispatch_key = f"dispatch_area_{dispatch_area.upper()}"
    if dispatch_key in row:
        row[dispatch_key] = 1
    
    type_key = f"initial_type_{initial_type.upper()}"
    if type_key in row:
        row[type_key] = 1
    
    df = pd.DataFrame([row])
    # predict response time in seconds
    
    
    
    prediction_seconds = model.predict(df)[0]
    prediction_minutes = round(float(prediction_seconds) / 60, 2)
    return prediction_minutes

In [39]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "predict_response_time",
            "description": "Predicts NYC 911 EMS response time in minutes based on call details and conditions.",
            "parameters": {
                "type": "object",
                "properties": {
                    "season":       {"type": "string", "enum": ["winter", "spring", "summer", "fall"]},
                    "borough":      {"type": "string", "description": "BROOKLYN, MANHATTAN, QUEENS, RICHMOND / STATEN ISLAND"},
                    "dispatch_area":{"type": "string", "description": "e.g. M1, K3, Q2, B4"},
                    "initial_type": {"type": "string", "description": "Call type code e.g. CARD, TRAUMA, EDP, SICK"},
                    "hour":         {"type": "integer", "description": "Hour of day 0-23"},
                    "day":          {"type": "integer", "description": "Day of month"},
                    "month":        {"type": "integer", "description": "Month 1-12"},
                    "is_weekend":   {"type": "integer", "enum": [0, 1]},
                    "is_holiday":   {"type": "integer", "enum": [0, 1]},
                    "is_rush_hour": {"type": "integer", "enum": [0, 1]},
                    "initial_severity": {"type": "integer", "description": "Severity level"},
                    "zipcode":      {"type": "integer"},
                    "temperature":  {"type": "number", "description": "Temperature in Fahrenheit"},
                    "precipitation":{"type": "number"},
                    "windspeed":    {"type": "number"},
                    "weathercode":  {"type": "integer"},
                    "closest_station_manhattan_miles": {"type": "number"}
                },
                "required": ["season", "borough", "dispatch_area", "initial_type",
                             "hour", "day", "month", "is_weekend", "is_holiday",
                             "is_rush_hour", "initial_severity", "zipcode",
                             "temperature", "precipitation", "windspeed",
                             "weathercode", "closest_station_manhattan_miles"]
            }
        }
    }
]

In [40]:
system_prompt = """
You are an expert data science assistant specializing in NYC 911 EMS response time analysis, the date of start date is 2024-09-01 and end date is 2025-08-31.

You have access to a trained ensemble of 4 seasonal machine learning models (winter, spring, summer, fall) 
built with scikit-learn that predict EMS response times in seconds based on:
- Location: borough, dispatch area, zipcode, distance from Manhattan
- Time: hour, day, month, rush hour, weekend, holiday
- Call type: initial_type codes (e.g. CARD, TRAUMA, EDP, SICK)
- Severity: initial_severity
- Weather: temperature, precipitation, windspeed, weathercode
- Operational flags: special events, standby, held

When making predictions:
- Always state the result in minutes AND provide context (e.g. fast/average/slow relative to typical ranges)
- Mention which seasonal model was used and why it matters

When explaining results to a technical audience:
- Reference feature importance, model behavior, and seasonal differences where relevant
- Be specific about what factors are likely driving the prediction

When generating reports or summaries:
- Switch to clear, concise language accessible to any audience
- Highlight actionable insights

Always ask for missing inputs conversationally — start with season, borough, call type, and hour, 
then gather remaining details naturally. Never dump a list of 10 questions at once.

"When asked general questions like 'which season is best', run the prediction function 
across all 4 seasons using typical average values (e.g. hour=14, Manhattan borough, 
SICK call type, weekday, no special conditions) and compare the results automatically 
without asking the user for input."
"""

In [41]:
# conversation history
conversation_history = []

def chat(user_message):
    conversation_history.append({"role": "user", "content": user_message})
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": system_prompt},*conversation_history],
        tools=tools,
        tool_choice="auto")
    message = response.choices[0].message
    ## If GPT wants to call your model, it will respond with a message like:
    if message.tool_calls:
        conversation_history.append(message)
        
        for tool_call in message.tool_calls:
            args = json.loads(tool_call.function.arguments)
            result = predict_response_time(**args)
            
            conversation_history.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": f"Predicted response time: {result} minutes"})
        
        # GPT explains the result
        follow_up = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "system", "content": system_prompt}] + conversation_history
        )
        assistant_message = follow_up.choices[0].message.content
    else:
        assistant_message = message.content
    
    conversation_history.append({"role": "assistant", "content": assistant_message})
    return assistant_message


In [42]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Exiting chat.")
        break
    reply = chat(user_input)
    print(f"Bot: {reply}\n")

Bot: Yes, rain can affect EMS response times. Precipitation impacts road conditions, visibility, and traffic flow, which can slow down emergency vehicles. To analyze the impact more precisely, the weather-related features such as precipitation, windspeed, and weathercode are used in our models to predict response times. If you have a specific scenario in mind, we could explore how rain might affect the prediction for that case. 

Bot: Yes, weather does affect EMS response times. Adverse weather conditions such as rain, snow, or strong winds can impact road conditions and visibility, making it harder for emergency vehicles to reach their destinations quickly. Our prediction models take into account weather-related features like temperature, precipitation, windspeed, and a weather code to assess their impact on response times accurately. 

If you have a particular scenario in mind, I can demonstrate how various weather conditions might affect the predicted response time for that situatio

c:\Users\ayman\AppData\Local\Programs\Python\Python311\Lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [ ]:
# predict the amount of calls for diiferent borough 
# r2 square score for times siers sarimma x 
# Time series model — looks at patterns over time (trends, seasonality, cycles)
# great at answering "what will average response times look like next month?"
# doesn't use features like borough or call type — just historical time patterns
# ----    IGNORE    ----
# respond with details on injury severity, weather conditions, and time of day to provide context for the prediction.
# respond with ems times are needed the most during rush hour on weekdays in Manhattan, especially for severe cardiac calls, and that bad weather can further increase response times.
# make a little  filtered data frame i used grouped by == table shows that average response time by hour location. 
# sampled month to month
